In [1]:
import pandas as pd
import numpy as np

In [6]:
# genes id, symbol and width infomation
info = pd.read_csv("/mnt/d/dailywork/01_Smear_seq/cancer/eso_squamou_cancer/geo/gene_id_symbol_width.tsv", sep="\t")
print(info.head())
# a example data
data = pd.read_csv("/mnt/d/dailywork/01_Smear_seq/cancer/eso_squamou_cancer/geo/GSE226192_raw_counts_GRCh38.p13_NCBI.tsv", sep="\t")
data.head()

     gene_id   width   symbol
1          1   17574     A1BG
2         10   14908     NAT2
3        100   67357      ADA
4       1000  245068     CDH2
5  100008586   16941  GAGE12F


,GeneID,GSM7067662,GSM7067663,GSM7067664,GSM7067665,GSM7067666,GSM7067667,GSM7067668,GSM7067669
0,100287102,13,6,15,11,12,3,16,15
1,653635,495,525,619,454,452,478,304,541
2,102466751,22,10,26,21,12,17,11,21
3,107985730,1,2,1,0,1,0,1,2
4,100302278,0,0,0,1,0,0,0,0


In [9]:
# check 
print(data.shape)
data.drop_duplicates(subset=['GeneID'], inplace=True)
print(data.shape)

(39376, 9)
(39376, 9)


In [15]:
data.rename(columns={'GeneID': 'gene_id'}, inplace=True)

In [ ]:
merge_info = pd.merge(data, info, on="gene_id")
print(merge_info.shape)
merge_info.head()

(29127, 11)


,gene_id,GSM7067662,GSM7067663,GSM7067664,GSM7067665,GSM7067666,GSM7067667,GSM7067668,GSM7067669,width,symbol
0,100287102,13,6,15,11,12,3,16,15,2541,DDX11L1
1,653635,495,525,619,454,452,478,304,541,15167,WASH7P
2,102466751,22,10,26,21,12,17,11,21,68,MIR6859-1
3,107985730,1,2,1,0,1,0,1,2,1556,MIR1302-2HG
4,100302278,0,0,0,1,0,0,0,0,138,MIR1302-2


In [33]:
# filter counts less than 5
merge_info = merge_info[merge_info.drop(['gene_id', 'width'], axis=1).sum(numeric_only=True, axis=1) > 5]
print(merge_info.shape)
merge_info.head()

(25285, 11)


,gene_id,GSM7067662,GSM7067663,GSM7067664,GSM7067665,GSM7067666,GSM7067667,GSM7067668,GSM7067669,width,symbol
0,100287102,13,6,15,11,12,3,16,15,2541,DDX11L1
1,653635,495,525,619,454,452,478,304,541,15167,WASH7P
2,102466751,22,10,26,21,12,17,11,21,68,MIR6859-1
3,107985730,1,2,1,0,1,0,1,2,1556,MIR1302-2HG
5,645520,1,0,0,0,4,2,0,3,1528,FAM138A


In [37]:
len(merge_info.symbol) == len(set(merge_info.symbol))

True

In [49]:
# calculate TPM

rpk = merge_info[[col for col in merge_info.columns if "GSM" in col]].div(merge_info['width'], axis=0)
rpk.index = merge_info.symbol
rpk.index.name = None
print(rpk.shape)
rpk.head()

(25285, 8)


,GSM7067662,GSM7067663,GSM7067664,GSM7067665,GSM7067666,GSM7067667,GSM7067668,GSM7067669
DDX11L1,0.005116,0.002361,0.005903,0.004329,0.004723,0.001181,0.006297,0.005903
WASH7P,0.032637,0.034615,0.040812,0.029933,0.029802,0.031516,0.020044,0.035670
MIR6859-1,0.323529,0.147059,0.382353,0.308824,0.176471,0.250000,0.161765,0.308824
MIR1302-2HG,0.000643,0.001285,0.000643,0.000000,0.000643,0.000000,0.000643,0.001285
FAM138A,0.000654,0.000000,0.000000,0.000000,0.002618,0.001309,0.000000,0.001963


In [58]:
tpm = rpk.T.div(rpk.sum(), axis=0).T * 10e6

In [60]:
tpm.to_csv('/mnt/d/dailywork/01_Smear_seq/cancer/eso_squamou_cancer/geo/tpm_without_log.tsv', sep='\t')